# Geometry calibration notebook.
## Use this after masking and setting proper ADU threshes for the azav and the the Jungfrau Sum. 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage as ndi
import xrayscatteringtools as xrst
xrst.enable_underscore_cleanup() # Registers post sell hook that deletes user-defined vairables that start with '_' (but not ipy magic variables)

### Importing data. Edit the run number, you may need to edit some of the keys. Usually is should work find so long as there is nothing special going on.
The main key that we work with is the `'Sums/jungfrau4M_calib_threshADU'` key. You can define your own in the producer with the `sumAlgo` kwarg of `det.storeSum()` and appropriate kwargs to `det.processSums()`. The key takeway is that any number after `'thresADU'` will be a __lower bound__ in ADU for thresholding.

In [ ]:
###############################################
runNumbers = []
folders = xrst.get_data_paths(runNumbers) # Defaults to the info in config.yaml. You can overwrite this with strings, character arrays, or lists of either.
###############################################
# (1) keys_to_combine: some keys loaded for each shot & stored per shot 
# (2) keys_to_sum: some keys loaded per each run and added 
# (3) keys_to_check : check if some keys exist and have same values in all runs and load these keys 
_keys_to_combine = [
    'ebeam/photon_energy',
]
_keys_to_sum = [
    'Sums/jungfrau4M_calib_thresADU2.5', # <- This key may need to be changed to reflect the ADU threshold set in the producer
    # 'Sums/jungfrau4M_calib_dg2low_threshADU5' # <- Any user-defined sums can go here too.
]
_keys_to_check = [
    # 'UserDataCfg/jungfrau4M/azav_mask0__azav_mask0_userMask', # <- Any masks for different azav regions can go here. We will combine them in the notebook
    # 'UserDataCfg/jungfrau4M/azav_mask1__azav_mask1_userMask',
    'UserDataCfg/jungfrau4M/cmask', # <- Combined mask for this run stored in the datastream.
    'UserDataCfg/jungfrau4M/x',
    'UserDataCfg/jungfrau4M/y',
]
##### Load the data in #####
_data = xrst.combineRuns(runNumbers, folders, _keys_to_combine, _keys_to_sum, _keys_to_check, verbose=False)  # this is the function to load the data with defined keys
############################

# userMaskFiltered = _data['UserDataCfg/jungfrau4M/azav_mask0__azav_mask0_userMask'].astype(bool) # User mask for this region
# userMask = _data['UserDataCfg/jungfrau4M/azav_mask1__azav_mask1_userMask'].astype(bool) # User mask for this region
jungfrau_sum = _data['Sums/jungfrau4M_calib_thresADU2.5']  # Total Jungfrau detector counts summed in a run
x, y = _data['UserDataCfg/jungfrau4M/x'], _data['UserDataCfg/jungfrau4M/y'] # coordinates of Jungfrau detector pixels x,y in micron
cmask = _data['UserDataCfg/jungfrau4M/cmask'].astype(bool) # Combineds mask
xray_keV = _data['ebeam/photon_energy'] # x-ray energy energy in keV

## Now that the data is loaded, let's prep the mask a little more in case it missed anything.
Use the `_binary_dilation_amount` to binary dialate the mask. If you did a good job on the masking, then you probably won't need to use this.

`_use_pdiff_masking` and `_pdiff_mask_limits` are for you to use the percent difference image (generated later on) to remove specific pixels which are obvious outliers. For the first time going through this, it won't do anything.

In [ ]:
############################
mask = np.copy(cmask) # & usermask 
_binary_dilation_amount = 0 # If zero, the dialated mask is the same as the cmask. Turn this number up if you can't see the scattering in the Combined mask from .h5 * J4M Sum plot
## Edit these after you run throguh the geometry calibration once
_use_pdiff_masking = False
_pdiff_mask_limits = [-20, 20] # Lower and upper bounds in percent for the pdiff
############################
if 'pdiff_img' in locals() and _use_pdiff_masking:
    print('Using pdiff_img for masking')
    _pdiff_mask = np.ones_like(mask)
    _pdiff_mask[~np.isnan(pdiff_img)] = (pdiff_img[~np.isnan(pdiff_img)] > _pdiff_mask_limits[0]) * (pdiff_img[~np.isnan(pdiff_img)] < _pdiff_mask_limits[1])
    mask *= _pdiff_mask
else:
    print('Not using pdiff_img for masking')
# Preforming a binary opening to mask out groups of bad pixels together.
if _binary_dilation_amount:
    mask = ndi.binary_opening(mask, structure=np.ones((1, _binary_dilation_amount, _binary_dilation_amount), dtype=bool)) & mask
plt.figure(figsize=[14,12])
plt.subplot(2,2,1)
xrst.plot_j4m(cmask)
plt.title('Combined mask from .h5')
plt.subplot(2,2,2)
xrst.plot_j4m(mask)
plt.title('Mask for calibration')
plt.subplot(2,2,3)
pcm = xrst.plot_j4m(jungfrau_sum*cmask)
plt.colorbar(pcm)
plt.title('Combined mask from .h5 * J4M Sum')
plt.subplot(2,2,4)
pcm = xrst.plot_j4m(jungfrau_sum*mask)
plt.colorbar(pcm)
plt.title('Mask for calibration * J4M Sum')
plt.show()
print(f'Percentage Masked (Combined mask from .h5): {100 * np.sum(~cmask)/np.size(cmask):.2f}%')
print(f'Percentage Masked (Mask for calibration): {100 * np.sum(~mask)/np.size(mask):.2f}%')

### One nice big plot to show the final image (masked) before geometry optimization

In [ ]:
plt.figure(figsize=[18,14])
_jungfrau_sum = np.copy(jungfrau_sum)
_jungfrau_sum[~mask] = np.nan
pcm = xrst.plot_j4m(_jungfrau_sum)
plt.colorbar(pcm)
plt.title('Masked Jungfrau Sum (big)')
plt.show()

### Importing or loading the theory pattern to fit this data to.
For data inside the xrayscatteringtools module, the docstring will contain more information about that pattern. e.g `xrst.theory.patterns.SF6__CCSD__aug_cc_pVDZ?`

In [ ]:
###################################################
system = xrst.theory.patterns.SF6__CCSD__aug_cc_pVDZ
theory_q, theory_I_q = system.q, system.I_q
###################################################
plt.plot(theory_q,theory_I_q)
plt.title(f'Theory {system.molecule} Pattern, {system.method}/{system.basis_set}')
plt.xlabel(r'q ($\AA^{-1}$)')
plt.ylabel(r'I(q) (Thompson Units)')
plt.show()

### Now to run the geometry calibration.
`run_geometry_calibration()` Has many parameters which are not used here. Run `xrst.calib.run_geometry_calibration?` for more information.

In [ ]:
##################################################
keV = xrst.get_config_for_runs(runNumbers, 'photon_energy','energy') # Auto with config.yaml
# keV = 9.666 # Override for photon energy if needed
##################################################
# Setting up a dict for cleaner usage.
_kwargs = {  
    'raw_image': jungfrau_sum,
    'x': x,
    'y': y,
    'mask': mask,
    'theory_q': theory_q,
    'theory_Iq': theory_I_q,
    'photon_energy_keV': keV,
}
fit, popt, pcov = xrst.calib.run_geometry_calibration(**_kwargs)
print(f'Converged Parameters: \n amplitude: {popt[0]:.2f}\n x0: {popt[1]:.2f}\n y0: {popt[2]:.2f}\n z0: {popt[3]:.2f}')

## Taking a look at the resultant fit, comparing to theory.

In [ ]:
# Setting up a dict for cleaner usage.
_kwargs = {
    'x': x,
    'y': y,
    'x0': popt[1],
    'y0': popt[2],
    'z0': popt[3],
    'mask': ~mask,
    'keV': keV,
}
# Taking the azav for comparison.
q, azav_exp = xrst.azimuthalBinning(jungfrau_sum,**_kwargs)
q, azav_fit = xrst.azimuthalBinning(fit,**_kwargs)
# Calculate percent difference
with np.errstate(invalid='ignore'):
    pdiff = 100*(azav_exp-azav_fit)/azav_fit
# Plotting
plt.figure(figsize=[18,5])
plt.subplot(1,3,1)
plt.plot(q,azav_exp,label='Experiment')
plt.plot(q,azav_fit,label='Fit')
plt.legend()
plt.title('Fitted Scattering Pattern')
plt.xlabel(r'q ($\AA^{-1}$)')
plt.ylabel(r'I(q)')
plt.subplot(1,3,2)
plt.plot(q,azav_exp*q,label='Experiment')
plt.plot(q,azav_fit*q,label='Fit')
plt.legend()
plt.title('Fitted Scattering Pattern')
plt.xlabel(r'q ($\AA^{-1}$)')
plt.ylabel(r'I(q)*q')
plt.subplot(1,3,3)
plt.plot(q,pdiff)
plt.axhline(0,linestyle='--',color='r')
plt.title('Percent Difference')
plt.ylabel('%')
plt.xlabel(r'q ($\AA^{-1}$)')
plt.show()

### Looking at the difference between the full images:

In [ ]:
#############
_vlimsPdiff = 30
#############
pdiff_img = 100*(jungfrau_sum-fit)/fit
pdiff_img[~mask] = np.nan

plt.figure(figsize=[18,5])

# --- Plot 1: Experiment ---
plt.subplot(1,3,1)
_jungfrau_sum = np.copy(jungfrau_sum)
_jungfrau_sum[~mask] = np.nan
pcm = xrst.plot_j4m(_jungfrau_sum, vmin=0)
plt.colorbar(pcm)
plt.title('Experiment')

ax = plt.gca()
ax.set_axis_off()

# --- Plot 2: Fit ---
plt.subplot(1,3,2)
_fit = np.copy(fit)
_fit[~mask] = np.nan
pcm = xrst.plot_j4m(_fit, vmin=0)
plt.colorbar(pcm)
plt.title('Fit')

ax = plt.gca()
ax.set_axis_off()

# --- Plot 3: Percent Difference ---
plt.subplot(1,3,3)
pcm = xrst.plot_j4m(pdiff_img, vmin=-_vlimsPdiff, vmax=_vlimsPdiff, cmap='RdBu_r')
plt.colorbar(pcm)
plt.title('Percent Difference Image')

ax = plt.gca()
ax.set_axis_off()

plt.show()

### At this point, you can go back up to the mask prep step and use the percent difference image to help out with the masking and re-do the calibration.

### Remember, when pasting the x0, y0, and z0 parameters into the producer, the z0 parameters should be in mm, not micron. Below are the pasteable correct parameters:

In [ ]:
print(f"func_dict['center'] = [{popt[1]}, {popt[2]}]")
print(f"func_dict['dis_to_sam'] = {popt[3]/1000}")